In [60]:
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
import tensorflow as tf
from tqdm import tqdm
import matplotlib.pyplot as plt
import coremltools as ct

In [69]:
data_augmentation = keras.Sequential([

    layers.experimental.preprocessing.RandomFlip("horizontal"),
    layers.experimental.preprocessing.RandomRotation(0.1),
    layers.experimental.preprocessing.RandomContrast((0,1))

])

def partionner(tab):
    n= tab.shape[0]
    y,x = np.zeros(n),np.zeros((n,256,256,3))
    for i in tqdm(range(len(tab))) :
        y[i],x[i,:,:,:] = tab[i,1],tab[i,0]
    return x,y


Found 1861 files belonging to 3 classes.
<BatchDataset shapes: ((None, 256, 256, 3), (None,)), types: (tf.float32, tf.int32)>


In [86]:
def init_CNN(directory, epochs = 2, classes = False, class_name = None, to_coreML=False):
    if classes : categories, m_name= {"handle" :0, "ball_bearing" : 1, "motor" : 2}, "incepNet_Casc_class"
    elif class_name == "handle" : categories, m_name = {"handle_lock": 3, "handle_lockless": 4}, "incepNet_Casc_label"+class_name
    elif class_name == "motor" : categories, m_name = {"motor_S" : 0, "motor_m" : 1}, "incepNet_Casc_label"+class_name

    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
    directory,
    labels="inferred",
    label_mode="int", shuffle=True, batch_size=32)

    print("Data is imported. Beginning of augmentation...")
    augmented_train_ds = train_ds.map(lambda x,y : (data_augmentation(x),y))
    augmented_train_ds = augmented_train_ds.prefetch(buffer_size=32)
    augmented_train_ds.shuffle(1000)
    print("Augmentation is done. Importation of InceptionV3 beginning...")
    base_model = tf.keras.applications.InceptionV3(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=None,
    pooling=None,
    classes=1000,
    classifier_activation="softmax")

    base_model.trainable = False


    print("Compilation of the CNN")
    input_shape = (256,256,3)
    inputs = keras.Input(shape=input_shape)
    x = tf.keras.layers.experimental.preprocessing.Rescaling(1.0/255.)(inputs)
    x = base_model(x, training=False)
    x = keras.layers.GlobalAveragePooling2D()(x)

    outputs = keras.layers.Dense(len(categories),activation=tf.nn.softmax)(x)
    model = keras.Model(inputs, outputs)

    model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    print("_________________ Here is the architecture of the CNN_________________")
    model.summary()

    print("Now begins training...")
    model.fit(augmented_train_ds, epochs=epochs)

#hep ----------> to implement fine tuning in the model initialisation process
 #   if fine_tuning :
#
  #      base_model.trainable = True
 #       model.summary()


    if to_coreML :
        print("Model is trained, now is converting the model into a CoreML format")

        class_labels = list(categories.keys())

        image_input = ct.ImageType(shape=(1, 256, 256, 3,),
                               bias=[-1,-1,-1])

        classifier_config = ct.ClassifierConfig(class_labels)

        your_model =  ct.convert(
        model, inputs=[image_input], classifier_config=classifier_config)

        print("Saving the mlmodel")
        m_name= m_name+".mlmodel"
        your_model.save(m_name)

    print("done")
    return model

In [ ]:
directory = "/Users/lucas/test_framer/class/"
model = init_CNN(directory=directory, classes = True)

In [ ]:
directory = "/Users/lucas/test_framer/label/"
classes = ["motor","handle"]
for classe in classes :
    directory_ = directory+classe
    print(directory_)
    init_CNN(directory=directory_, classes = False, class_name=classe, epochs= 4)